In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

In [9]:
# !pip install kora -q
# from kora import drive
# drive.link_nbs()a

     |████████████████████████████████| 57 kB 3.5 MB/s 
Mounted at /content/drive


In [ ]:
!pip install import-ipynb
import import_ipynb

In [5]:
%cd "/gdrive/MyDrive/Colab Notebooks/"

/gdrive/MyDrive/Colab Notebooks


In [12]:
#@title Test for type
# Due to errors in importing, I copied the model here as well.


# ##############################
# below is the code for the inception block
from torch.nn.modules.batchnorm import BatchNorm2d
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()

        assert out_channels%8==0, 'out channel should be mutiplier of 8'

        ################################
        ## P4.1. Write your code here ##
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels,out_channels//4,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//4),
            nn.ReLU()
        )
        self.block_2= nn.Sequential(
            nn.Conv2d(in_channels,out_channels//2,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//2),
            nn.ReLU(),
            nn.Conv2d(out_channels//2,out_channels//2, stride=1,padding=1,kernel_size=3,bias=False),
            BatchNorm2d(out_channels//2),
            nn.ReLU()
        )
        self.block_3 = nn.Sequential(
            nn.Conv2d(in_channels,out_channels//8,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU(),
            nn.Conv2d(out_channels//8,out_channels//8, stride=1,padding=2,kernel_size=5,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU()
        )
        self.max_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            nn.Conv2d(in_channels,out_channels//8,kernel_size=1,padding=0,stride=1,bias=False),
            BatchNorm2d(out_channels//8),
            nn.ReLU()
        )


        ################################

    def forward(self, x):

        ################################
        ## P4.2. Write your code here ##
        output1= self.block_1(x)
        output2= self.block_2(x)
        output3= self.block_3(x)
        output4= self.max_pool(x)
        # print("output1 size {}, output2 size {}, output3 size {}, output4 size {}, ".format(output1.size(),output2.size(),output3.size(),output4.size()))
        output = torch.cat( (self.block_1(x),self.block_2(x),self.block_3(x),self.max_pool(x)), 1 )


 
        return output
        ################################
# Below is our code for the inception model, which consists of inception blocks defined above and other layers

from torch.nn.modules.flatten import Flatten
from torch.nn.modules.pooling import MaxPool2d
class MyNetwork(nn.Module):
    def __init__(self, nf, block_type='inception', num_blocks=[1,1, 1, 1,1]):
        super(MyNetwork, self).__init__()

        
        self.block_type = block_type

        # Define blocks according to block_type
        if self.block_type == 'inception':
            block = InceptionBlock
            block_args = lambda x: (x, x)

        # Define block layer by stacking multiple blocks. 
        self.block_00 = nn.Sequential(*[block(*block_args(nf)) for _ in range(num_blocks[0])])
        self.block0 = nn.Sequential(*[block(*block_args(nf*2)) for _ in range(num_blocks[1])])
        self.block1 = nn.Sequential(*[block(*block_args(nf*4)) for _ in range(num_blocks[2])])
        self.block2 = nn.Sequential(*[block(*block_args(nf*4)) for _ in range(num_blocks[3])])
        self.block3 = nn.Sequential(*[block(*block_args(nf*4)) for _ in range(num_blocks[4])])

        ################################

        self.pre_block_00 = nn.Sequential(
            nn.Conv2d(3,nf,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        
        )

        self.pre_block0 = nn.Sequential(
            nn.Conv2d(nf,nf*2,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        
        )

        self.pre_block1 = nn.Sequential(
            nn.Conv2d(nf*2,nf*4,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.pre_block2 = nn.Sequential(
            nn.Conv2d(nf*4,nf*4,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.pre_block3 = nn.Sequential(
            nn.Conv2d(nf*4,nf*4,stride=1,kernel_size=3,padding=1,bias=False),
            nn.BatchNorm2d(nf*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.final = nn.Sequential(
            nn.AdaptiveAvgPool2d( 1 ),
            nn.Flatten(),
            nn.Linear( nf*4 , 8)
        )


        ################################

    def forward(self, x):
        # the forward function
        #######################################################################
        first_out = self.block_00(self.pre_block_00(x))
        output_0 = self.block0(self.pre_block0(first_out))
        output1_1=self.pre_block1(output_0)
        output1= self.block1(output1_1)
        output2=self.block2(self.pre_block2(output1))
        output3=self.block3(self.pre_block3(output2))
        output = self.final(output3)
        # print("size of tensor after final stage: ",output.size())

 
        return output
        #######################################################################




# ##############################
# import Damage_type_self_inception
num_filters=16
num_blocks = [5,5,5,5,5]
model_type = MyNetwork(num_filters, 'inception', num_blocks)
device = ("cuda" if torch.cuda.is_available() else "cpu")
model_path = '/gdrive/MyDrive/project/type_inception.pt'
# model_loaded = model.load_state_dict(torch.load(model_path))

model_type.load_state_dict(torch.load(model_path))
model_type.cuda()
model_type.eval()

import torchvision.transforms as transforms

def type_preprocess(img):
  test_transform = transforms.Compose([
    transforms.CenterCrop(128),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

  return test_transform(img)
import urllib

from PIL import Image
from urllib.request import Request, urlretrieve  # Python 3

def type_test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = type_preprocess(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_type(img)
      # print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Prediction is: Broken headlamp '
      elif output[0].argmax()==1:
        return ' Prediction is: Broken tail lamp '
      elif output[0].argmax()==2:
        return ' Prediction is: glass shatter '
      elif output[0].argmax()==3:
        return ' Prediction is: door scratch '
      elif output[0].argmax()==4:
        return ' Prediction is: door dent '
      elif output[0].argmax()==5:
        return ' Prediction is: bumper dent '
      elif output[0].argmax()==6:
        return ' Prediction is: bumper scratch '
      else:
        return ' Prediction is: Unknown damge type '



In [51]:
#@title Test for severity
import import_ipynb
import Damage_severity_self_inception

num_filters=16
num_blocks = [5,5,5,5,5]
model_severity = Damage_severity_self_inception.MyNetwork(num_filters, 'inception', num_blocks)

device = ("cuda" if torch.cuda.is_available() else "cpu")
model_path = '/gdrive/MyDrive/project/severity_inception.pt'
# model_loaded = model.load_state_dict(torch.load(model_path))

model_severity.load_state_dict(torch.load(model_path))
model_severity.cuda()
model_severity.eval()

import torchvision.transforms as transforms

def severity_preprocess(img):
  test_transform = transforms.Compose([
    transforms.CenterCrop(128),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

  return test_transform(img)

import urllib

from PIL import Image
from urllib.request import Request, urlretrieve  # Python 3

def severity_test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = severity_preprocess(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_severity(img)
      # print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Minor damage '
      elif output[0].argmax()==1:
        return ' Prediction is: Moderate damage '
      else:
        return ' Prediction is: Severe damage '



In [48]:
#@title Test for location_cifar
import import_ipynb

import Damage_location_self_inception

model_loc_cifar = Damage_location_self_inception.newnetwork(num_classes=3).to(device)
model_path = '/gdrive/MyDrive/project/car_damage_location/self_inception/test_inception.pt'
# model_loaded = model.load_state_dict(torch.load(model_path))

model_loc_cifar.load_state_dict(torch.load(model_path))
model_loc_cifar.cuda()
model_loc_cifar.eval()

location_test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(32),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])
def location_preprocess(img):
  img_processed = location_test_transform(img)
  return img_processed

# test
import urllib

from PIL import Image
from urllib.request import Request, urlretrieve  # Python 3

def test_loc_cifar(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = location_preprocess(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)

    print('We are going to predict the location of damage with second location model:')
    with torch.no_grad():
      output = model_loc_cifar(img)
      # print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Front damage '
      elif output[0].argmax()==1:
        return ' Prediction is: Side damage '
      else:
        return ' Prediction is: Rear damage '


In [ ]:
#@title Test for location_imagenet
import import_ipynb
device = torch.device("cuda")

import damage_location_2

transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

# test
import urllib
import torch
from PIL import Image
device = ("cuda" if torch.cuda.is_available() else "cpu")

def preprocess(img):
  img = transform(img)

  return img


path = '/gdrive/MyDrive/project/car_damage_location/model_location_weights_final_2.pth'
model_loc_imagenet = damage_location_2.inceptionv3()

model_loc_imagenet.load_state_dict(torch.load(path))
model_loc_imagenet.to(device)

import urllib

from urllib.request import Request, urlretrieve  # Python 3

def test_loc_imagenet(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = transform(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)
    model_loc_imagenet.eval()
    print('We are going to predict the location of damage with first location model:')
    with torch.no_grad():
      output = model_loc_imagenet(img)
      # print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Front damage '
      elif output[0].argmax()==1:
        return ' Prediction is: Side damage '
      else:
        return ' Prediction is: Rear damage '







In [ ]:
#@title Test for damaged
import import_ipynb
import Car_damaged_or_not
device = torch.device("cuda")


class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=1):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=True)
        self.inception.dropout=nn.Dropout(0.3)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.inception(images)
        features_2 = self.dropout(features)
        return self.sigmoid(features_2).squeeze(1)


model_damaged = CNN()
model_path = '/gdrive/MyDrive/project/car_damaged_or_not/model_weights.pth'
model_damaged.load_state_dict(torch.load(model_path))

model_damaged.load_state_dict(torch.load(model_path))
model_damaged.cuda()
model_damaged.eval()

import torchvision.transforms as transforms
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

# test
import urllib
import torch
from PIL import Image
device = ("cuda" if torch.cuda.is_available() else "cpu")


from urllib.request import Request, urlretrieve  # Python 3

def damaged_test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = transform(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)

    print('We are going to predict whether it is damaged!')
    with torch.no_grad():
      output = model_damaged(img)
      # print(output)
      if output[0] > 0.45 and output[0] < 0.55:
        return "We are not sure about the car's condition"
      elif output[0] > 0.55:
        return ' The car seems to be whole. Are you sure it is damaged? '
      else:
        return ' The car is damaged! We are certain in the damage '
    



In [ ]:
#@title Test for car

!pip install efficientnet_pytorch

torch.backends.cudnn.benchmark = True

!pip install git+https://github.com/lucasb-eyer/pydensecrf.git
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils
import json
path = '/gdrive/MyDrive/project/'
import urllib
from efficientnet_pytorch import EfficientNet
model_car = EfficientNet.from_pretrained('efficientnet-b4')
from urllib.request import Request, urlretrieve  # Python 3
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/img.jpg
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]
# list of top classes
top_classes = json.load( open( path+"car_detection.json" ) )

def car_preprocess(image):
  transform  = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
  processed_image = transform(image).unsqueeze(0)
  return processed_image

def car_categories(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    img = car_preprocess(Image.open('save.jpg'))
    model_car.eval()
    with torch.no_grad():
      output = model_car(img)
    top_preds = torch.topk(output, k=3).indices.squeeze(0).tolist()
    result = [ (labels_map[index], torch.softmax(output,dim=1)[0,index].item() ) for index in top_preds]

    print ('Module 1: We are going to predict whether this is a car...')
    for category in result:
        # print(top)
        if category[0] in top_classes:
            print ('Our model has detected: ',category[0], 'with percentage:', category[1])
            return "The model predicts this is a car"
    return "Not a car picture. Please try again."



In [65]:
def full_test(image_path):
  url = image_path
  car = car_categories(url)
  if car != 'The model predicts this is a car':
    return 'Testing stops here'
  else:
    print(car)
  print('\n Going to the next test. Is the car Damaged? ')
  
  damaged = damaged_test(url)
  if damaged == ' The car seems to be whole. Are you sure it is damaged? ':
    return 'The car is not damaged. Testing stops here.' 
  else:
    print(damaged)
  print('\n Going to the next test.Location of Damage.')

  location_1 = test_loc_imagenet(url)
  print(location_1)
  location_2 = test_loc_cifar(url)
  print(location_2)
  print('\nGoing to the next test. Severity of Damage:')

  severity = severity_test(url)
  print(severity)
  print('\n Going to the last test.Type of Damage:')

  type_damage = type_test(url)
  print(type_damage)


In [66]:
url = 'https://cdn.motor1.com/images/mgl/N0n7Y/s1/2019-hyundai-elantra-sport-review.jpg'
full_test(url)

Module 1: We are going to predict whether this is a car...
Our model has detected:  car wheel with percentage: 0.5786836743354797
The model predicts this is a car

 Going to the next test. Is the car Damaged? 
We are going to predict whether it is damaged!


'The car is not damaged. Testing stops here.'